In [1]:
import os
from pathlib import Path
from collections import namedtuple, deque
import numpy as np

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day24.txt'


class Point(namedtuple("Point", ['x', 'y'])):
    
    def moves(self):
        moves = (
            Point(self.x+1, self.y),
            Point(self.x, self.y+1),
            Point(self.x-1, self.y),
            Point(self.x, self.y-1)   
        )
        yield from (n for n in moves if 0 <= n.x < WIDTH and 0 <= n.y < HEIGHT)
        

# Part One

In [2]:
with open(FOLDER / in_file) as f:
    raw = np.array([list(line.strip()) for line in f])

HEIGHT, WIDTH =  raw[1:-1, 1:-1].shape
  
start_x = np.flatnonzero(raw[0] == '.')[0]
end_x = np.flatnonzero(raw[-1] == '.')[0]

p = Point(start_x - 1, -1)
end = Point(end_x - 1, raw.shape[0] -3)


class Storms():
    def __init__(self, raw):
        self.EW = (raw == '>')
        self.WE = raw == '<'
        self.NS = raw == 'v'
        self.SN = raw == '^'

    def step_blizzards(self):
        self.EW = np.roll(self.EW, 1, axis=1)
        self.WE = np.roll(self.WE, -1, axis=1)
        self.NS = np.roll(self.NS, 1, axis=0)
        self.SN = np.roll(self.SN, -1, axis=0)

    def merge_blizzards(self):
        return ~np.any([self.EW, self.WE, self.NS, self.SN], axis=0)

blizzards = Storms(raw[1:-1, 1:-1])



## How many map state are there?

It looks like 600! Maybe cache them so we can prune the search?

In [3]:
maps = set([tuple(blizzards.merge_blizzards().flatten())])
for i in range(2000):
    blizzards.step_blizzards()
    maps.add(tuple(blizzards.merge_blizzards().flatten()))
    
len(maps)

600

In [4]:
blizzards = Storms(raw[1:-1, 1:-1])


def BFS(blizzards, p, end):    
    q = deque([(p, 0)])
    current_count = -1
    
    chart = blizzards.merge_blizzards()
    
    map_state = tuple(chart.flatten())

    # track state so we don't keep trying the same 
    # state with a worse time
    best_so_far = {(map_state, p): 0}    
    
    while q:
        move, count = q.popleft()
        if count != current_count:
            # step blizzard at each level of BFS
            # e.i. when the depth count changes
            blizzards.step_blizzards()
            chart = blizzards.merge_blizzards()
            map_state = tuple(chart.flatten())
            current_count = count  
            
        if move == end:
            # end is the spot adjacent to the end
            # it's easier than including the off-map
            # point when looking at neighbors. Just add 1
            return count + 1
        
        for step in move.moves():
            if chart[step.y, step.x]:
                best = best_so_far.get((map_state, step))
                if best is None or count < best:
                    best_so_far[(map_state, step)] = count
                    q.append((step, count+1))

        # we can hunker down in place if we want
        if move == (0, -1) or chart[move.y, move.x]:
            best = best_so_far.get((map_state, move))
            if best is None or count < best:
                best_so_far[(map_state, move)] = count
                q.append((move, count+1))
        
BFS(blizzards, p, end)


271

# Part 2

For part two, keep track of whether we have touched the end and the start.
Adjust the cache to cache:
• map state (flattened array)
• position
• whether we've touched the end once
• whether we've touched the start again.

In [5]:
blizzards = Storms(raw[1:-1, 1:-1])
before_start = Point(0, 0)

def BFS(blizzards, p, end):    
    q = deque([(p, 0, False, False)])
    current_count = -1
    
    chart = blizzards.merge_blizzards()
    
    map_state = tuple(chart.flatten())

    # Add two booleans to the cache to know whether we've 
    # touched the ends
    # Maybe we don't need to search the way back again? 
    best_so_far = {(map_state, p, False, False): 0}    
    
    while q:
        move, count, touched_end, touched_start = q.popleft()
        
        if count != current_count:
            # step blizzard at each level of BFS
            # e.i. when the depth count changes
            blizzards.step_blizzards()
            chart = blizzards.merge_blizzards()
            map_state = tuple(chart.flatten())
            current_count = count  
        
        if move == before_start and touched_end:
            true_start = Point(move.x, move.y-1)
            best = best_so_far.get((map_state, true_start, touched_end, touched_start))
            if best is None or count < best:
                best_so_far[(map_state, step, touched_end, touched_start)] = count
                q.append((true_start, count+1, touched_end, True))

        
        if move == end:
            if touched_start:
                return count + 1
            true_end = Point(move.x, move.y+1)
            
            best = best_so_far.get((map_state, true_end, touched_end, touched_start))

            if best is None or count < best:
                best_so_far[(map_state, step, touched_end, touched_start)] = count
                q.append((true_end, count+1, True, False))
        
        for step in move.moves():
            if chart[step.y, step.x]:
                best = best_so_far.get((map_state, step, touched_end, touched_start))
                
                if best is None or count < best:
                    best_so_far[(map_state, step, touched_end, touched_start)] = count
                    q.append((step, count+1, touched_end, touched_start))


        if move == (0, -1) or move == (end.x, end.y+1) or chart[move.y, move.x]:
            best = best_so_far.get((map_state, move, touched_end, touched_start))

            if best is None or count < best:
                best_so_far[(map_state, move, touched_end, touched_start)] = count
                q.append((move, count+1, touched_end, touched_start))
        
BFS(blizzards, p, end)


813

In [6]:
p

Point(x=0, y=-1)